In [1]:
'''
generate dataset from qplib
from Xi Gao
'''

import osqp
import time
import tqdm
import random
import pickle
import numpy as np
import gurobipy as gp

from gurobipy import GRB
from scipy.sparse import csc_matrix

## Import Data

In [2]:
num_var = 5223
num_ineq = 106
num_eq = 732

# qp_path = ''
m = gp.read('QPLIB_8906.lp')

Read LP format model from file QPLIB_8906.lp
Reading time = 0.01 seconds
obj: 838 rows, 5223 columns, 15558 nonzeros


In [3]:
#quadratic matrix
ini_q_matrix = m.getQ().toarray()

#linear vector
ini_p_vec = []
variables = m.getVars()
for var in variables:
    ini_p_vec.append(var.obj)
ini_p_vec = np.array(ini_p_vec)

print('The shape of quadratic matrix: {}.'.format(ini_q_matrix.shape))
print('The shape of coefficient vector: {}.'.format(ini_p_vec.shape))

The shape of quadratic matrix: (5223, 5223).
The shape of coefficient vector: (5223,).


In [4]:
#inequality and equality coefficient matrix
ini_ineq_matrix = []
ini_ineq_rhs = []
ini_eq_matrix = []
ini_eq_rhs = []
a_matrix = m.getA().toarray()

i = 0
for constr in m.getConstrs():
    if constr.sense == '<':
        ini_ineq_matrix.append(a_matrix[i,:])
        ini_ineq_rhs.append(constr.RHS)
        i += 1
    elif constr.sense == '>':
        ini_ineq_matrix.append(-a_matrix[i,:])
        ini_ineq_rhs.append(-constr.RHS)
        i += 1
    elif constr.sense == '=':
        ini_eq_matrix.append(a_matrix[i,:])
        ini_eq_rhs.append(constr.RHS)
        i += 1

ini_ineq_matrix = np.array(ini_ineq_matrix)
ini_ineq_rhs = np.array(ini_ineq_rhs)
ini_eq_matrix = np.array(ini_eq_matrix)
ini_eq_rhs = np.array(ini_eq_rhs)

print('The shape of inequality coefficient matrix: {}.'.format(ini_ineq_matrix.shape))
print('The shape of inequality RHS vector: {}.'.format(ini_ineq_rhs.shape))
print('The shape of equality coefficient matrix: {}.'.format(ini_eq_matrix.shape))
print('The shape of equality RHS vector: {}.'.format(ini_eq_rhs.shape))

The shape of inequality coefficient matrix: (106, 5223).
The shape of inequality RHS vector: (106,).
The shape of equality coefficient matrix: (732, 5223).
The shape of equality RHS vector: (732,).


In [5]:
ini_ineq_rhs.shape

(106,)

## Distribution

In [6]:
unique_values, value_counts = np.unique(ini_q_matrix, return_counts=True)
q_v_p = []
# 打印结果
print('Quadratic Matrix:')
for value, count in zip(unique_values, value_counts):
    if value==0.0:
        print("Value:", value, "Count:", count)
    else:
        print("Value:", value, "Count:", count, "Proportion:", count/len(np.nonzero(ini_q_matrix)[0]))
        q_v_p.append([value, count/len(np.nonzero(ini_q_matrix)[0])])
q_v_p = np.array(q_v_p)

Quadratic Matrix:
Value: 0.0 Count: 27222754
Value: 0.5 Count: 54835 Proportion: 0.9624396665204037
Value: 1.0 Count: 90 Proportion: 0.0015796401930671348
Value: 2.0 Count: 109 Proportion: 0.0019131197893813075
Value: 5.0 Count: 1914 Proportion: 0.03359368143922773
Value: 20.0 Count: 17 Proportion: 0.000298376480912681
Value: 25.0 Count: 10 Proportion: 0.00017551557700745942


In [7]:
unique_values, value_counts = np.unique(ini_p_vec, return_counts=True)
# 打印结果
print('Coefficient Vector:')
for value, count in zip(unique_values, value_counts):
    print("Value:", value, "Count:", count)

Coefficient Vector:
Value: 7.434 Count: 10
Value: 7.56 Count: 2
Value: 7.907293 Count: 1
Value: 8.0442565 Count: 1
Value: 8.258583 Count: 1
Value: 8.358 Count: 2
Value: 8.718234 Count: 1
Value: 8.85 Count: 7
Value: 9.0 Count: 1
Value: 9.558 Count: 7
Value: 9.72 Count: 1
Value: 9.95 Count: 1
Value: 10.746 Count: 1
Value: 11.172 Count: 4
Value: 13.3 Count: 2
Value: 13.806 Count: 4
Value: 13.9637645 Count: 1
Value: 14.04 Count: 1
Value: 14.1593925 Count: 1
Value: 14.172924 Count: 1
Value: 14.364 Count: 2
Value: 15.522 Count: 1
Value: 20.178 Count: 5
Value: 20.52 Count: 1
Value: 20.5313925 Count: 1
Value: 20.532 Count: 14
Value: 20.636655 Count: 1
Value: 20.748 Count: 2
Value: 20.88 Count: 2
Value: 20.886 Count: 5
Value: 21.2393925 Count: 1
Value: 21.24 Count: 1
Value: 21.344655 Count: 1
Value: 22.686 Count: 1
Value: 23.084 Count: 2
Value: 23.268 Count: 2
Value: 23.364 Count: 3
Value: 23.482 Count: 1
Value: 23.76 Count: 1
Value: 23.837293 Count: 1
Value: 23.9742565 Count: 1
Value: 24.072 C

In [8]:
unique_values, value_counts = np.unique(ini_ineq_matrix, return_counts=True)
# 打印结果
print('Inequality Matrix:')
for value, count in zip(unique_values, value_counts):
    print("Value:", value, "Count:", count)

Inequality Matrix:
Value: -1.6 Count: 4
Value: -1.143 Count: 2
Value: -1.066 Count: 8
Value: -1.0 Count: 125
Value: -0.0200312 Count: 2
Value: -0.01 Count: 4
Value: -0.00833125 Count: 14
Value: -0.00714375 Count: 4
Value: -0.00666875 Count: 2
Value: -0.0066625 Count: 8
Value: -0.00625 Count: 4
Value: 0.0 Count: 548307
Value: 0.00625 Count: 4
Value: 0.0066625 Count: 8
Value: 0.00666875 Count: 2
Value: 0.00714375 Count: 4
Value: 0.00833125 Count: 14
Value: 0.01 Count: 4
Value: 0.0200312 Count: 2
Value: 1.0 Count: 5102
Value: 1.066 Count: 8
Value: 1.143 Count: 2
Value: 1.6 Count: 4


In [9]:
unique_values, value_counts = np.unique(ini_ineq_rhs, return_counts=True)
# 打印结果
print('Inequality RHS Vector:')
for value, count in zip(unique_values, value_counts):
    print("Value:", value, "Count:", count)

Inequality RHS Vector:
Value: -4.770012 Count: 1
Value: -4.188446 Count: 1
Value: -3.298332 Count: 1
Value: -2.7944 Count: 1
Value: -2.568468 Count: 1
Value: -2.117949 Count: 1
Value: -2.095802 Count: 1
Value: -1.963334 Count: 1
Value: -1.649166 Count: 1
Value: -1.6407481 Count: 1
Value: -1.220513 Count: 1
Value: -0.946586 Count: 1
Value: -0.91731 Count: 1
Value: -0.916223 Count: 1
Value: -0.733848 Count: 1
Value: -0.706785 Count: 2
Value: -0.6986 Count: 1
Value: -0.5888179 Count: 1
Value: -0.558881 Count: 1
Value: -0.523556 Count: 1
Value: -0.502564 Count: 1
Value: -0.348187 Count: 1
Value: -0.3295335 Count: 1
Value: -0.3274212 Count: 1
Value: -0.315529 Count: 1
Value: -0.1155604 Count: 1
Value: -0.0963003 Count: 1
Value: -0.0960343 Count: 1
Value: -0.0619808 Count: 1
Value: -0.0413206 Count: 1
Value: -0.0248705 Count: 1
Value: -0.0124352 Count: 1
Value: 0.0 Count: 68
Value: 2.0 Count: 1
Value: 3.0 Count: 1
Value: 5.0 Count: 1
Value: 30.0 Count: 2


In [10]:
unique_values, value_counts = np.unique(ini_eq_matrix, return_counts=True)
# 打印结果
print('Equality Matrix:')
for value, count in zip(unique_values, value_counts):
    print("Value:", value, "Count:", count)

Equality Matrix:
Value: -1.0 Count: 5100
Value: 0.0 Count: 3813009
Value: 1.0 Count: 5127


In [11]:
unique_values, value_counts = np.unique(ini_eq_rhs, return_counts=True)
# 打印结果
print('Equality RHS Vector:')
for value, count in zip(unique_values, value_counts):
    print("Value:", value, "Count:", count)

Equality RHS Vector:
Value: 0.0 Count: 201
Value: 0.00918473 Count: 1
Value: 0.0103301 Count: 5
Value: 0.0124352 Count: 1
Value: 0.0183695 Count: 1
Value: 0.0186528 Count: 1
Value: 0.0192601 Count: 1
Value: 0.0206603 Count: 3
Value: 0.0248705 Count: 2
Value: 0.0275542 Count: 1
Value: 0.0309904 Count: 4
Value: 0.0310881 Count: 5
Value: 0.0373057 Count: 4
Value: 0.0383828 Count: 1
Value: 0.0413206 Count: 3
Value: 0.0435233 Count: 4
Value: 0.0480172 Count: 1
Value: 0.0497409 Count: 1
Value: 0.0516507 Count: 2
Value: 0.0577802 Count: 4
Value: 0.0619808 Count: 3
Value: 0.0621762 Count: 2
Value: 0.0631057 Count: 1
Value: 0.0683938 Count: 1
Value: 0.0717949 Count: 2
Value: 0.0720257 Count: 1
Value: 0.072311 Count: 1
Value: 0.0746114 Count: 4
Value: 0.0770403 Count: 2
Value: 0.080829 Count: 1
Value: 0.0826411 Count: 2
Value: 0.0870466 Count: 3
Value: 0.0895599 Count: 1
Value: 0.0929712 Count: 3
Value: 0.0963003 Count: 6
Value: 0.0994819 Count: 1
Value: 0.103301 Count: 2
Value: 0.105699 Count: 

## Random Sample

### Randomly sample the variables, ineq cons and eq cons to perturb

In [11]:
# num_chosed_var = 1500
# # num_chosed_ineq = 100
# num_chosed_eq = 200

# chosed_var = sorted(random.sample(range(0, num_var), num_chosed_var))
# # chosed_ineq = sorted(random.sample(range(0, num_ineq), num_chosed_ineq))
# chosed_eq = sorted(random.sample(range(0, num_eq), num_chosed_eq))

In [12]:
num_chosed_var = 5223
num_chosed_ineq = 106
num_chosed_eq = 732

# chosed_eq = sorted(random.sample(range(0, num_eq), num_chosed_eq))

In [13]:
#perturb quadratic matrix
# q_matrix = ini_q_matrix[chosed_var, :]
# q_matrix = q_matrix[:, chosed_var]

q_matrix = ini_q_matrix.copy()


#perturb coefficient vector
p_vec = ini_p_vec.copy()
# nonzero_indices = np.nonzero(ini_p_vec)
# nonzero_values = ini_p_vec[nonzero_indices]
# perturbed_values = nonzero_values * (1 +  np.random.uniform(-0.1, 0.1, size=nonzero_values.shape))
# p_vec[nonzero_indices] = perturbed_values
# p_vec = p_vec[chosed_var]

#perturb coefficient vector
ineq_matrix = ini_ineq_matrix.copy()
# nonzero_indices = np.nonzero(ini_ineq_matrix)
# nonzero_values = ini_ineq_matrix[nonzero_indices]
# perturbed_values = nonzero_values * (1 +np.random.uniform(-0.1, 0.1, size=nonzero_values.shape))
# ineq_matrix[nonzero_indices] = perturbed_values
# ineq_matrix = ineq_matrix[chosed_ineq, :]
# ineq_matrix = ineq_matrix[:, chosed_var]

#perturb coefficient vector
ineq_rhs = ini_ineq_rhs.copy()
# nonzero_indices = np.nonzero(ini_ineq_rhs)
# nonzero_values = ini_ineq_rhs[nonzero_indices]
# perturbed_values = nonzero_values * (1 + np.random.uniform(-0.1, 0.1, size=nonzero_values.shape))
# ineq_rhs[nonzero_indices] = perturbed_values
# ineq_rhs = ineq_rhs[chosed_ineq]

#perturb coefficient vector
eq_matrix = ini_eq_matrix.copy()
# nonzero_indices = np.nonzero(ini_eq_matrix)
# nonzero_values = ini_eq_matrix[nonzero_indices]
# perturbed_values = nonzero_values * (1 +np.random.uniform(-0.1, 0.1, size=nonzero_values.shape))
# eq_matrix[nonzero_indices] = perturbed_values
# eq_matrix = eq_matrix[chosed_eq, :]
# eq_matrix = eq_matrix[:, chosed_var]

#perturb coefficient vector
eq_rhs = ini_eq_rhs.copy()
nonzero_indices = np.nonzero(ini_eq_rhs)
nonzero_values = ini_eq_rhs[nonzero_indices]
perturbed_values = nonzero_values * (1 + np.random.uniform(-0.1, 0.1, size=nonzero_values.shape))
eq_rhs[nonzero_indices] = perturbed_values
# eq_rhs = eq_rhs[chosed_eq]

In [14]:
def perturb_eq_rhs(ini_eq_rhs, num_samples):
    eq_rhs_list = []
    for _ in range(num_samples):
        eq_rhs = ini_eq_rhs.copy()
        nonzero_indices = np.nonzero(ini_eq_rhs)
        nonzero_values = ini_eq_rhs[nonzero_indices]
        perturbed_values = nonzero_values * (1 + np.random.uniform(-0.1, 0.1, size=nonzero_values.shape))
        eq_rhs[nonzero_indices] = perturbed_values
        eq_rhs_list.append(eq_rhs)
    return np.array(eq_rhs_list)

In [22]:
model = gp.Model()

variables = []
# add variables 
for i in range(num_chosed_var):
    var_name = "x{}".format(i + 2)
    var = model.addVar(vtype=GRB.CONTINUOUS, name=var_name)
    variables.append(var)
    
#set objectives
obj_expr = gp.QuadExpr()
for i in range(num_chosed_var):
    obj_expr += p_vec[i]*variables[i]
    for j in range(num_chosed_var):
        obj_expr += q_matrix[i][j] * variables[i] * variables[j]
model.setObjective(obj_expr, GRB.MINIMIZE)

#add constraints
model.addConstrs(gp.LinExpr(ineq_matrix[i, :num_chosed_var], variables) <= ineq_rhs[i] for i in range(num_chosed_ineq))
model.addConstrs(gp.LinExpr(eq_matrix[i, :num_chosed_var], variables) == eq_rhs[i] for i in range(num_chosed_eq))


# 求解模型
model.optimize()

# 打印结果
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print("Objective value: ", model.objVal)
else:
    print("Optimization failed. Status:", model.status)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 838 rows, 5223 columns and 15558 nonzeros
Model fingerprint: 0xf6b72197
Model has 56975 quadratic objective terms
Coefficient statistics:
  Matrix range     [6e-03, 2e+00]
  Objective range  [7e+00, 6e+03]
  QObjective range [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-03, 3e+01]
Presolve removed 201 rows and 1048 columns
Presolve time: 0.01s
Presolved: 637 rows, 4175 columns, 10505 nonzeros
Presolved model has 44855 quadratic objective terms
Ordering time: 0.01s

Barrier statistics:
 Free vars  : 1670
 AA' NZ     : 1.016e+05
 Factor NZ  : 1.524e+05 (roughly 5 MB of memory)
 Factor Ops : 1.262e+07 (less than 1 second per iteration)
 Threads    : 24

                  Objective                Residu

## Generate Data

In [15]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


num_examples = 10000

np.random.seed(17)
Q = q_matrix
p = p_vec
A = eq_matrix
X = perturb_eq_rhs(ini_eq_rhs, num_examples)
G = ineq_matrix
h = ineq_rhs
problem = SimpleProblem(Q, p, A, G, h, X)
# problem.calc_Y()
# print(len(problem.Y))

In [16]:
with open("./random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
        pickle.dump(problem, f)

-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 5223, constraints m = 838
          nnz(P) + nnz(A) = 72533
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -4.8288e+09   1.08e+01   9.90e+06   1.00e-01   1.73e-02s
  25  -1.0000e+30   1.77e-01   1.27e+03   1.00e-01   2.41e-02s

status:               dual infeasible
num

## Gurobi v.s. OSQP

## Random generate data

In [33]:
import gurobipy as gp

from gurobipy import Model, GRB, QuadExpr, LinExpr
from scipy.sparse import csc_matrix
import numpy as np
import osqp

num_var = 200
num_ineq = 100
num_eq = 100

num_chosed_var = 200
num_chosed_ineq = 100
num_chosed_eq = 100

model = gp.Model()
q_matrix = np.diag(np.random.random(num_var))
p_vec = np.random.random(num_var)
eq_matrix = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
eq_rhs = np.random.uniform(-1, 1, size=(num_eq))
ineq_matrix = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
ineq_rhs = np.sum(np.abs(ineq_matrix@np.linalg.pinv(eq_matrix)), axis=1)

## osqp

In [32]:
Q = q_matrix
p = p_vec
Xi = eq_rhs
A = eq_matrix
G = ineq_matrix
h = ineq_rhs

solver = osqp.OSQP()
my_A = np.vstack([A, G])
my_l = np.hstack([Xi, -np.ones(h.shape[0]) * np.inf])
my_u = np.hstack([Xi, h])
solver.setup(P=csc_matrix(Q), q=p, A=csc_matrix(my_A), l=my_l, u=my_u, verbose=True, eps_prim_inf=1e-4)
results = solver.solve()
print(results.x)

-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 200, constraints m = 200
          nnz(P) + nnz(A) = 40200
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.9877e+01   9.86e-01   4.55e+02   1.00e-01   1.12e-02s
[ 1.02650603 -2.35624769 -0.76669684 -0.87381019 -0.40741345  0.00924522
 -0.46455934 -0.76055459 -0.93453

## Gurobi

In [30]:
# Create a new Gurobi model
model = Model("qp")

# Add variables to the model
n_vars = Q.shape[0]
vars = []
for i in range(n_vars):
    vars.append(model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name=f"x_{i}"))

# Set the objective function
obj = QuadExpr()
for i in range(n_vars):
    for j in range(n_vars):
        if Q[i, j] != 0:
            obj.add(vars[i] * vars[j] * Q[i, j])

for i in range(n_vars):
    if p[i] != 0:
        obj.add(vars[i] * p[i])

model.setObjective(obj, GRB.MINIMIZE)

# Add equality constraints
for i in range(A.shape[0]):
    expr = LinExpr()
    for j in range(n_vars):
        if A[i, j] != 0:
            expr.add(vars[j] * A[i, j])
    model.addConstr(expr == Xi[i])

# Add inequality constraints
for i in range(G.shape[0]):
    expr = LinExpr()
    for j in range(n_vars):
        if G[i, j] != 0:
            expr.add(vars[j] * G[i, j])
    model.addConstr(expr <= h[i])

model.setParam('FeasibilityTol', 1e-4)
# Solve the model
model.optimize()

# Retrieve the results
if model.status == GRB.OPTIMAL:
    results = [var.x for var in vars]
    print("Optimal solution:", results)
else:
    print("No optimal solution found.")


Set parameter FeasibilityTol to value 0.0001
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 200 rows, 200 columns and 40000 nonzeros
Model fingerprint: 0x8211ae18
Model has 200 quadratic objective terms
Coefficient statistics:
  Matrix range     [8e-08, 4e+00]
  Objective range  [2e-03, 1e+00]
  QObjective range [7e-04, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-02, 1e+01]
Presolve time: 0.01s
Presolved: 200 rows, 200 columns, 40000 nonzeros
Presolved model has 200 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 200
 AA' NZ     : 1.990e+04
 Factor NZ  : 2.010e+04
 Factor Ops : 2.687e+06 (less than 1 second per iteration)
 Threads    : 24

                  Objective                Residual
Iter       Primal          

   1   1.65607711e+06 -1.67011537e+06  1.09e+02 1.36e+00  1.67e+05     0s
   2   7.85445712e+05 -7.97759240e+05  7.90e-01 1.28e-02  3.24e+04     0s
   3   1.16221322e+05 -1.21005448e+05  8.36e-12 1.28e-08  4.74e+03     0s
   4   1.64707339e+04 -1.82732826e+04  3.57e-12 7.70e-13  6.95e+02     0s
   5   2.20921052e+03 -2.86776409e+03  1.56e-12 1.68e-13  1.02e+02     0s
   6   2.52913467e+02 -4.77784440e+02  5.06e-13 3.72e-14  1.46e+01     0s
   7   6.77761970e+00 -7.75870805e+01  1.88e-13 5.48e-14  1.69e+00     0s
   8  -1.53703190e+01 -2.84330487e+01  6.21e-14 2.58e-14  2.61e-01     0s
   9  -1.84199370e+01 -1.94056187e+01  1.50e-14 4.47e-15  1.97e-02     0s
  10  -1.86627126e+01 -1.87053990e+01  2.74e-15 7.28e-15  8.54e-04     0s
  11  -1.86774975e+01 -1.86784486e+01  1.13e-15 5.48e-15  1.90e-05     0s
  12  -1.86779927e+01 -1.86780566e+01  2.00e-15 3.22e-15  1.28e-06     0s
  13  -1.86780174e+01 -1.86780187e+01  1.09e-15 5.11e-15  2.45e-08     0s
  14  -1.86780176e+01 -1.86780176e+01 